In [1]:
from __future__ import print_function
import time
import cv2
import numpy as np
import seaborn as sns; sns.set()
import os
import dlib
from PIL import Image
from skimage import exposure
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from IPython.display import clear_output
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
sc = StandardScaler()

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

Using TensorFlow backend.


In [3]:
def detect_faces(image):

    # Create a face detector
    face_detector = dlib.get_frontal_face_detector()

    # Run detector and get bounding boxes of the faces on image.
    detected_faces = face_detector(image, 1)
    face_frames = [(x.left(), x.top(),
                    x.right(), x.bottom()) for x in detected_faces]

    return face_frames

def contraste(imagen):
    clahe = cv2.createCLAHE(clipLimit=3., tileGridSize=(8, 8))
    lab = cv2.cvtColor(imagen, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    l2 = clahe.apply(l)
    lab = cv2.merge((l2, a, b))
    contraste = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    return contraste


In [5]:
# Import data
path_imagen = "/home/pzampella/NimSet/CropWhiteBackground/"
path_archivo = "/home/pzampella/NimSet/NimStim_ratings.csv"
data = np.genfromtxt(path_archivo, dtype=None, delimiter=',')

# Inputs
new_dim = 32
classes = 7

nombres = []
x = []
x_flipped = []
y = []

for i in range(0, len(data)):
    clear_output()
    print(str(i+1)+"/"+str(len(data)))
    if os.path.exists(path_imagen + data[i][0].upper()):
        nombres.append(data[i][0].upper())
        aux0 = cv2.imread(path_imagen + data[i][0].upper(), True)
        aux1 = cv2.cvtColor(contraste(aux0), cv2.COLOR_BGR2GRAY)
        detected_faces = detect_faces(aux1)
        if len(detected_faces) > 0:
            face = np.array(Image.fromarray(aux1).crop(detected_faces[0]))
            contrast = exposure.equalize_hist(face) * 255
            Aux3 = sc.fit_transform(cv2.resize(contrast, (new_dim, new_dim)))
            Aux3_flipped = np.flip(Aux3, 1)
            aux = np.concatenate(Aux3)
            aux_flipped = np.concatenate(Aux3_flipped)
            x.append(aux)
            x.append(aux_flipped)
            y.append(data[i][1])
            y.append(data[i][1])

y = np.array(y)
x =  np.array(x)
y = y.reshape(-1,1)
ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

x_train = np.reshape(x_train, (x_train.shape[0], new_dim, new_dim, 1))
x_test = np.reshape(x_test, (x_test.shape[0], new_dim, new_dim, 1))

print("Data imported!")

657/657
Data imported!


In [4]:
#Image augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    samplewise_center=True,
    fill_mode='nearest')

datagen.fit(x_train, augment=True)

In [11]:
# Network Parameters
n_input = new_dim*new_dim # data input (img shape: 640*490)
n_classes = 7 # total classes (7 emotions: 0=anger, 1=neutral, 2=disgust, 3=fear, 4=happy, 5=sadness, 6=surprise)
#np.random.seed(27)
classifier= Sequential()

classifier.add(Convolution2D(16, (3, 3), activation="relu", input_shape=x_train.shape[1:]))
classifier.add(MaxPooling2D(pool_size =(2,2)))

classifier.add(Flatten())

classifier.add(Dense(units=128, activation="relu"))
classifier.add(Dense(units=n_classes, activation="sigmoid"))

classifier.compile(optimizer="adam", loss="categorical_crossentropy",
                   metrics=["accuracy"])

#Network training
epochs = 200
batch = 104

#classifier.fit(x_train, y_train, batch_size = batch, epochs = epochs)
classifier.fit_generator(datagen.flow(x_train, y_train, batch_size=batch), steps_per_epoch= x_train.shape[0]//batch, epochs=epochs)

Epoch 1/200
10/10 [==============================] - 4s 360ms/step - loss: 1.6849 - acc: 0.3998
Epoch 2/200
10/10 [==============================] - 2s 208ms/step - loss: 1.3133 - acc: 0.4979
Epoch 3/200
10/10 [==============================] - 2s 239ms/step - loss: 1.4121 - acc: 0.4059
Epoch 4/200
10/10 [==============================] - 2s 202ms/step - loss: 1.3345 - acc: 0.4721
Epoch 5/200
10/10 [==============================] - 2s 172ms/step - loss: 1.1850 - acc: 0.5552
Epoch 6/200
10/10 [==============================] - 1s 130ms/step - loss: 1.2461 - acc: 0.4836
Epoch 7/200
10/10 [==============================] - 1s 78ms/step - loss: 1.1412 - acc: 0.5118
Epoch 8/200
10/10 [==============================] - 1s 65ms/step - loss: 1.3389 - acc: 0.5541
Epoch 9/200
10/10 [==============================] - 1s 82ms/step - loss: 1.1090 - acc: 0.5529
Epoch 10/200
10/10 [==============================] - 1s 68ms/step - loss: 0.9066 - acc: 0.6552
Epoch 11/200
10/10 [=======================

10/10 [==============================] - 3s 257ms/step - loss: 0.0376 - acc: 0.9933
Epoch 169/200
10/10 [==============================] - 2s 214ms/step - loss: 0.0261 - acc: 1.0000
Epoch 170/200
10/10 [==============================] - 3s 268ms/step - loss: 0.0382 - acc: 0.9932
Epoch 171/200
10/10 [==============================] - 2s 211ms/step - loss: 0.0278 - acc: 0.9971
Epoch 172/200
10/10 [==============================] - 2s 230ms/step - loss: 0.0260 - acc: 0.9990
Epoch 173/200
10/10 [==============================] - 3s 259ms/step - loss: 0.0281 - acc: 0.9962
Epoch 174/200
10/10 [==============================] - 2s 205ms/step - loss: 0.0204 - acc: 1.0000
Epoch 175/200
10/10 [==============================] - 2s 209ms/step - loss: 0.0246 - acc: 0.9961
Epoch 176/200
10/10 [==============================] - 2s 201ms/step - loss: 0.0288 - acc: 0.9961
Epoch 177/200
10/10 [==============================] - 2s 197ms/step - loss: 0.0311 - acc: 0.9971
Epoch 178/200
10/10 [=============

In [7]:
from keras.models import model_from_json
model_path1 = r'/home/pzampella/NimSet/CNN/Face/Keras/CNN_79_1534936535'
json_file = open(model_path1 + r'/model.json', 'r')
#Fin cargar archivos
loaded_model_json = json_file.read()
json_file.close()
classifier = model_from_json(loaded_model_json)
classifier.load_weights(model_path1 + r'/model.h5')
classifier.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [8]:
y_pred = classifier.predict(x_test)
#y_pred = (y_pred > 0.5)

count = 0
for i in range(0, y_pred.shape[0]):
    if np.argmax(y_pred[i]) == np.argmax(y_test[i]):
        count = count+1
    else:
        continue
print(str(count)+"/"+str(y_pred.shape[0]))
print("Accuracy: "+str(int(100*count/(x_test.shape[0])))+"%")

253/261
Accuracy: 96%


In [11]:
count0 = 0
count1 = 0
count2 = 0
count3 = 0
count4 = 0
count5 = 0
count6 = 0

total0 = 0
total1 = 0
total2 = 0
total3 = 0
total4 = 0
total5 = 0
total6 = 0

for i in range(0, y_pred.shape[0]):
    if np.argmax(y_pred[i]) == np.argmax(y_test[i]):
        if np.argmax(y_test[i]) == 0:
            count0 = count0 + 1
            total0 = total0 + 1
        if np.argmax(y_test[i]) == 1:
            count1 = count1 + 1
            total1 = total1 + 1
        if np.argmax(y_test[i]) == 2:
            count2 = count2 + 1
            total2 = total2 + 1
        if np.argmax(y_test[i]) == 3:
            count3 = count3 + 1
            total3 = total3 + 1
        if np.argmax(y_test[i]) == 4:
            count4 = count4 + 1
            total4 = total4 + 1
        if np.argmax(y_test[i]) == 5:
            count5 = count5 + 1
            total5 = total5 + 1
        if np.argmax(y_test[i]) == 6:
            count6 = count6 + 1
            total6 = total6 + 1
    else:
        if np.argmax(y_test[i]) == 0:
            total0 = total0 + 1
        if np.argmax(y_test[i]) == 1:
            total1 = total1 + 1
        if np.argmax(y_test[i]) == 2:
            total2 = total2 + 1
        if np.argmax(y_test[i]) == 3:
            total3 = total3 + 1
        if np.argmax(y_test[i]) == 4:
            total4 = total4 + 1
        if np.argmax(y_test[i]) == 5:
            total5 = total5 + 1
        if np.argmax(y_test[i]) == 6:
            total6 = total6 + 1
            
print("A: " + str(int(100*count0/(total0)))+"%")
print("B: " + str(int(100*count1/(total1)))+"%")
print("C: " + str(int(100*count2/(total2)))+"%")
print("D: " + str(int(100*count3/(total3)))+"%")
print("E: " + str(int(100*count4/(total4)))+"%")
print("F: " + str(int(100*count5/(total5)))+"%")
print("G: " + str(int(100*count6/(total6)))+"%")

A: 96%
B: 97%
C: 100%
D: 100%
E: 97%
F: 90%
G: 94%


In [13]:
# save model to JSON
folder = '/home/pzampella/NimSet/CNN/Face/Keras/CNN_' + str(int(100*count/(x_test.shape[0]))) + '_' + str(int(time.time())) + '/'
if not os.path.exists(folder):
    os.makedirs(folder)

model_json = classifier.to_json()
with open(folder + 'model.json', "w") as json_file:
    json_file.write(model_json)
# save weights to HDF5
classifier.save_weights(folder + 'model.h5')
print("Model saved to disk as" + folder)

Model saved to disk
